In [ ]:
!pip install --user pykafka

In [1]:
from pykafka import KafkaClient
client = KafkaClient(hosts="188.185.79.229:9092")

topic = client.topics['email_alert']

In [2]:
consumer = topic.get_simple_consumer()
for message in consumer:
    print(message.value)

KeyboardInterrupt: 

In [ ]:
!wget http://central.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.3.1/spark-streaming-kafka-0-8-assembly_2.11-2.3.1.jar

In [2]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.streaming.kafka import TopicAndPartition
import json

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
def sendingAlert(subject,count):
    return [subject,count,count>10]
spark.udf.register("sendingAlert", sendingAlert)

<function __main__.sendingAlert(subject, count)>

In [3]:
ssc = StreamingContext(sc, 5) #change frequency of getting data
ssc.checkpoint("_checkpoint")

In [4]:
kafkaStream =KafkaUtils.createDirectStream(ssc, ["email_alert"],
                                           {
                                               "metadata.broker.list":"188.185.79.229:9092",
                                               "auto.offset.reset":"smallest"
                                           }
                                          )

In [5]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [6]:
emails_dstream = parsed.map(lambda email: [email['subject'],email['timestamp'],1])\
                        .reduceByKeyAndWindow(lambda x,y: x+y, 10, 5)
# emails_dstream.pprint()
# benchmark=emails_dstream.map(lambda x:[x[0],x[1],x[1]>30])
benchmark=emails_dstream.map(lambda x: sendingAlert(x[0],x[1]))
# structuredOutput.foreachRDD(rdd => 
#   import sparkSession.implicits._
#   val df = rdd.toDF()
#   df.write.format("parquet").mode("append").save(s"$workDir/$targetFile")
# })
benchmark.pprint()
ssc.start()
ssc.awaitTerminationOrTimeout(10)


-------------------------------------------
Time: 2019-06-11 15:16:40
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:16:45
-------------------------------------------



False

-------------------------------------------
Time: 2019-06-11 15:16:50
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:16:55
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:17:00
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:17:05
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:17:10
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:17:15
-------------------------------------------

-------------------------------------------
Time: 2019-06-11 15:17:20
-------------------------------------------



In [7]:
ssc.stop()

-------------------------------------------
Time: 2019-06-11 15:17:25
-------------------------------------------



In [ ]:
!pip install --user kafka-python

In [ ]:
from kafka import KafkaConsumer
import json
try:
    print('Welcome to parse engine')
    consumer = KafkaConsumer('filter_alert',
                             bootstrap_servers='188.185.79.229:9092',
#To subscribe since the first message that has been produced to kafka producer
                             auto_offset_reset='earliest',
                             enable_auto_commit=False,
                             group_id=None,
                             value_deserializer=lambda m: json.loads(m)
                            )
    for message in consumer:
        print(message.value['subject'])
except Exception as e:
    print(e)
    # Logs the error appropriately.
    pass

Welcome to parse engine
